In [1]:
import os
print(os.environ['CONDA_DEFAULT_ENV'])


YZUP


In [2]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import re

In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Türkçe stop kelimeler
stop_words = set(stopwords.words('turkish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nezih\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
vocab_size = 10000
embedding_dim = 16
max_length = 250
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 60000


In [5]:
df = pd.read_csv('troff-v1.0.tsv', delimiter='\t')

# Metin sütununu küçük harfe çevirin
df['text'] = df['text'].str.lower()

# Noktalama işaretleri ve özel karakterleri kaldırın
df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Sayıları kaldırın
df['text'] = df['text'].apply(lambda x: re.sub(r'\d+', '', x))

# Gereksiz boşlukları kaldırın
df['text'] = df['text'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# Stop kelimeleri kaldırın
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# Yinelenen kayıtları kaldırın
df.drop_duplicates(subset=['text'], inplace=True)

df['label'] = df['label'].apply(lambda x: 0 if x == 'non' else 1)

In [6]:
len(df['label'].tolist())

35236

In [7]:
df2 = pd.read_csv('train.csv')

# Metin sütununu küçük harfe çevirin
df2['text'] = df2['text'].str.lower()

# Noktalama işaretleri ve özel karakterleri kaldırın
df2['text'] = df2['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Sayıları kaldırın
df2['text'] = df2['text'].apply(lambda x: re.sub(r'\d+', '', x))

# Gereksiz boşlukları kaldırın
df2['text'] = df2['text'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# Stop kelimeleri kaldırın
df2['text'] = df2['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# Yinelenen kayıtları kaldırın
df2.drop_duplicates(subset=['text'], inplace=True)

In [8]:
len(df2['label'].tolist())

42349

In [24]:
df3 = pd.read_csv('train2.csv')

df3 = df3[['tweet','label']]

df3.dropna(subset=['tweet'], inplace=True)

# Metin sütununu küçük harfe çevirin
df3['tweet'] = df3['tweet'].str.lower()

# Noktalama işaretleri ve özel karakterleri kaldırın
df3['tweet'] = df3['tweet'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Sayıları kaldırın
df3['tweet'] = df3['tweet'].apply(lambda x: re.sub(r'\d+', '', x))

# Gereksiz boşlukları kaldırın
df3['tweet'] = df3['tweet'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# Stop kelimeleri kaldırın
df3['tweet'] = df3['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# Yinelenen kayıtları kaldırın
df3.drop_duplicates(subset=['tweet'], inplace=True)

df3['label'] = df3['label'].apply(lambda x: 0 if x == 'hiçbiri' else 1)

df3.head(5)

,tweet,label
0,iyi partili türkkan milyon suriyeli gelecek,0
1,başkan erdoğandan suriyeli mülteci mesajı dest...,0
2,arslan bulut yazdı metrekarelik konaklar yaptı...,1
3,güzel bir amaçla yola çıkmış insan askerini po...,0
4,kimin parası türk halkının parası peki bizim k...,1


In [25]:
sentences = df['text'].tolist() + df2['text'].tolist() + df3['tweet'].tolist()
labels = df['label'].tolist() + df2['label'].tolist() + df3['label'].tolist()

In [26]:
len(labels)

78583

In [27]:
len(sentences)

78583

In [79]:
from sklearn.model_selection import train_test_split
training_sentences, testing_sentences, training_labels, testing_labels = train_test_split(sentences, labels, test_size=0.2, random_state=12)

In [80]:
print(f"Lenght training sentences {len(training_sentences)}, lenght testing sentences {len(testing_sentences)}")
print(f"Lenght training label {len(training_labels)}, lenght testing labels {len(testing_labels)}")

Lenght training sentences 62866, lenght testing sentences 15717
Lenght training label 62866, lenght testing labels 15717


In [81]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [82]:
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [102]:
def create_model(hp):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(units=hp.Int('neurons_layer_1', min_value=64, max_value=512, step=32),
                              activation=hp.Choice('activation_layer_1', values=['relu', 'tanh'])),
        tf.keras.layers.Dropout(rate=hp.Float('dropout_layer_1', min_value=0.2, max_value=0.5, step=0.1)),
        tf.keras.layers.Dense(units=hp.Int('neurons_layer_5', min_value=24, max_value=64, step=12),
                              activation=hp.Choice('activation_layer_2', values=['relu', 'tanh'])),
        tf.keras.layers.Dense(units=hp.Int('neurons_layer_2', min_value=24, max_value=64, step=12),
                              activation=hp.Choice('activation_layer_2', values=['relu', 'tanh'])),
        tf.keras.layers.Dropout(rate=hp.Float('dropout_layer_2', min_value=0.0, max_value=0.5, step=0.1)),
        tf.keras.layers.Dense(units=hp.Int('neurons_layer_3', min_value=16, max_value=32, step=16),
                              activation=hp.Choice('activation_layer_3', values=['relu', 'tanh'])),
        tf.keras.layers.Dense(units=hp.Int('neurons_layer_4', min_value=16, max_value=32, step=8),
                              activation=hp.Choice('activation_layer_2', values=['relu', 'tanh'])),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Get optimizer from hyperparameters
    optimizer=tf.keras.optimizers.Adam(learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model




In [20]:
!pip install keras-tuner

   ---------------------------------------- 0.0/129.1 kB ? eta -:--:--
   --------- ------------------------------ 30.7/129.1 kB 1.4 MB/s eta 0:00:01
   ------------ -------------------------- 41.0/129.1 kB 667.8 kB/s eta 0:00:01
   ---------------------------------- ----- 112.6/129.1 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 129.1/129.1 kB 1.1 MB/s eta 0:00:00


In [94]:
from kerastuner.tuners import RandomSearch, BayesianOptimization

In [105]:
tuner = RandomSearch(
    create_model,
    objective='val_accuracy',
    max_trials=10,
    directory='my_tuning_dir_rsearch',
    project_name='nlp_model_tuning'
)

In [107]:
tuner.search(training_padded, training_labels,
             epochs=10,
             validation_data=(testing_padded, testing_labels))

Trial 10 Complete [00h 01m 14s]
val_accuracy: 0.8898645043373108

Best val_accuracy So Far: 0.9023350477218628
Total elapsed time: 00h 13m 04s


In [44]:
pip install keras-tuner[bayesian]

Note: you may need to restart the kernel to use updated packages.


In [103]:
tuner_b = BayesianOptimization(
    create_model,
    objective='val_accuracy',
    max_trials=10,
    directory='my_tuning_dir_b',
    project_name='nlp_model_tuning'
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


Reloading Tuner from my_tuning_dir_b\nlp_model_tuning\tuner0.json


In [104]:
tuner_b.search(training_padded, training_labels, epochs=50, validation_data=(testing_padded, testing_labels), callbacks=[stop_early])

Trial 10 Complete [00h 01m 30s]
val_accuracy: 0.9012534022331238

Best val_accuracy So Far: 0.9037348031997681
Total elapsed time: 00h 12m 47s


In [108]:
best_hps_b = tuner_b.get_best_hyperparameters(num_trials=1)[0]

In [159]:
best_hps_r = tuner.get_best_hyperparameters(num_trials=1)[0]

In [110]:
model_b = tuner.hypermodel.build(best_hps_b)

In [160]:
model_r = tuner.hypermodel.build(best_hps_r)

In [114]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_b.fit(training_padded, training_labels, epochs=100, validation_data=(testing_padded, testing_labels), callbacks=[early_stopping])

Epoch 1/100
1965/1965 [==============================] - 9s 4ms/step - loss: 0.4118 - accuracy: 0.8200 - val_loss: 0.3326 - val_accuracy: 0.8723
Epoch 2/100
1965/1965 [==============================] - 9s 4ms/step - loss: 0.2740 - accuracy: 0.8961 - val_loss: 0.4016 - val_accuracy: 0.8631
Epoch 3/100
1965/1965 [==============================] - 8s 4ms/step - loss: 0.2403 - accuracy: 0.9105 - val_loss: 0.2900 - val_accuracy: 0.8894
Epoch 4/100
1965/1965 [==============================] - 8s 4ms/step - loss: 0.2212 - accuracy: 0.9192 - val_loss: 0.2846 - val_accuracy: 0.8948
Epoch 5/100
1965/1965 [==============================] - 9s 4ms/step - loss: 0.2070 - accuracy: 0.9259 - val_loss: 0.2768 - val_accuracy: 0.8983
Epoch 6/100
1965/1965 [==============================] - 8s 4ms/step - loss: 0.1936 - accuracy: 0.9304 - val_loss: 0.2948 - val_accuracy: 0.8972
Epoch 7/100
1965/1965 [==============================] - 9s 4ms/step - loss: 0.1844 - accuracy: 0.9345 - val_loss: 0.2966 - val_ac

In [161]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_r.fit(training_padded, training_labels, epochs=100, validation_data=(testing_padded, testing_labels), callbacks=[early_stopping])

Epoch 1/100
1965/1965 [==============================] - 8s 4ms/step - loss: 0.4268 - accuracy: 0.8100 - val_loss: 0.3639 - val_accuracy: 0.8558
Epoch 2/100
1965/1965 [==============================] - 7s 4ms/step - loss: 0.2789 - accuracy: 0.8939 - val_loss: 0.2861 - val_accuracy: 0.8909
Epoch 3/100
1965/1965 [==============================] - 7s 4ms/step - loss: 0.2416 - accuracy: 0.9092 - val_loss: 0.2791 - val_accuracy: 0.8924
Epoch 4/100
1965/1965 [==============================] - 7s 4ms/step - loss: 0.2176 - accuracy: 0.9210 - val_loss: 0.3050 - val_accuracy: 0.8895
Epoch 5/100
1965/1965 [==============================] - 7s 4ms/step - loss: 0.2051 - accuracy: 0.9252 - val_loss: 0.2950 - val_accuracy: 0.8857
Epoch 6/100
1965/1965 [==============================] - 7s 4ms/step - loss: 0.1936 - accuracy: 0.9288 - val_loss: 0.2765 - val_accuracy: 0.8999
Epoch 7/100
1965/1965 [==============================] - 7s 4ms/step - loss: 0.1816 - accuracy: 0.9326 - val_loss: 0.2889 - val_ac

In [183]:
sentence = ["Herkese Merhaba", "terörist grup öldürüldü"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model_b.predict(padded))
print(model_r.predict(padded))

1/1 [==============================] - 0s 22ms/step
[[0.07115555]
 [0.3904998 ]]
1/1 [==============================] - 0s 30ms/step
[[0.04762675]
 [0.27489117]]


In [163]:
y_pred_model_b = model_b.predict(testing_padded)
y_pred_model_r = model_r.predict(testing_padded)

492/492 [==============================] - 1s 1ms/step


In [179]:
y_pred_model_b_binary = [1 if eleman > 0.6 else 0 for eleman in y_pred_model_b]
y_pred_model_r_binary = [1 if eleman > 0.6 else 0 for eleman in y_pred_model_r]

In [180]:
print("Sınıflandırma Raporu model_b:")
print(classification_report(testing_labels, y_pred_model_b_binary))
print("Doğruluk:", accuracy_score(testing_labels, y_pred_model_b_binary))

Sınıflandırma Raporu model_b:
              precision    recall  f1-score   support

           0       0.89      0.97      0.93     10344
           1       0.93      0.76      0.84      5373

    accuracy                           0.90     15717
   macro avg       0.91      0.87      0.88     15717
weighted avg       0.90      0.90      0.90     15717

Doğruluk: 0.8984539034166825


In [181]:
print("Sınıflandırma Raporu model_r:")
print(classification_report(testing_labels, y_pred_model_r_binary))
print("Doğruluk:", accuracy_score(testing_labels, y_pred_model_r_binary))

Sınıflandırma Raporu model_r:
              precision    recall  f1-score   support

           0       0.89      0.97      0.93     10344
           1       0.92      0.77      0.84      5373

    accuracy                           0.90     15717
   macro avg       0.91      0.87      0.88     15717
weighted avg       0.90      0.90      0.90     15717

Doğruluk: 0.9001081631354584


In [132]:
model_b.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 250, 16)           160000    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_6 (Dense)             (None, 448)               7616      
                                                                 
 dropout_2 (Dropout)         (None, 448)               0         
                                                                 
 dense_7 (Dense)             (None, 60)                26940     
                                                                 
 dense_8 (Dense)             (None, 60)                3660      
                                                      

In [182]:
model_r.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 250, 16)           160000    
                                                                 
 global_average_pooling1d_2   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_12 (Dense)            (None, 256)               4352      
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_13 (Dense)            (None, 60)                15420     
                                                                 
 dense_14 (Dense)            (None, 36)                2196      
                                                      

In [134]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVC

In [151]:
svc_model = SVC(kernel='rbf', C=1.5, gamma='scale')
svc_model.fit(training_padded, training_labels)
y_pred_svc = svc_model.predict(testing_padded)

In [152]:
from sklearn.metrics import classification_report, accuracy_score
print("Sınıflandırma Raporu SVC:")
print(classification_report(testing_labels, y_pred_svc))
print("Doğruluk:", accuracy_score(testing_labels, y_pred_svc))

Sınıflandırma Raporu SVC:
              precision    recall  f1-score   support

           0       0.69      0.97      0.80     10344
           1       0.70      0.15      0.24      5373

    accuracy                           0.69     15717
   macro avg       0.69      0.56      0.52     15717
weighted avg       0.69      0.69      0.61     15717

Doğruluk: 0.6868995355347712


In [153]:
from sklearn.ensemble import RandomForestClassifier

In [154]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=15)
rf_model.fit(training_padded, training_labels)
y_pred_rf = rf_model.predict(testing_padded)

In [155]:
print("Sınıflandırma Raporu RF:")
print(classification_report(testing_labels, y_pred_rf))
print("Doğruluk:", accuracy_score(testing_labels, y_pred_rf))

Sınıflandırma Raporu RF:
              precision    recall  f1-score   support

           0       0.84      0.96      0.89     10344
           1       0.89      0.64      0.74      5373

    accuracy                           0.85     15717
   macro avg       0.86      0.80      0.82     15717
weighted avg       0.85      0.85      0.84     15717

Doğruluk: 0.8477444804988229


In [156]:
from sklearn.naive_bayes import MultinomialNB

In [157]:
nb_model = MultinomialNB()
nb_model.fit(training_padded, training_labels)
y_pred_nb = nb_model.predict(testing_padded)

In [158]:
print("Sınıflandırma Raporu NB:")
print(classification_report(testing_labels, y_pred_nb))
print("Doğruluk:", accuracy_score(testing_labels, y_pred_nb))

Sınıflandırma Raporu NB:
              precision    recall  f1-score   support

           0       0.69      0.77      0.73     10344
           1       0.44      0.35      0.39      5373

    accuracy                           0.63     15717
   macro avg       0.57      0.56      0.56     15717
weighted avg       0.61      0.63      0.61     15717

Doğruluk: 0.6261373035566584
